In [44]:
import numpy as np
import itertools
from beamforming import solve_beamforming_with_selected_antennas
from multiprocessing import Pool
from as_omar import as_omar
# from as_bb import solve_bb, DefaultBranchingPolicy
# from as_bb import ASBBenv as Environment

from as_bb_test import solve_bb, DefaultBranchingPolicy
from as_bb_test import ASBBenv as Environment
import os
import pickle
import time
from pathlib import Path
from antenna_selection.observation import Observation
from antenna_selection.model_tester import *


N = 12
M = 8
max_ant = 4
folder_path = '/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/validation_set/N={},M={},L={}_ftpl'.format(N,M,max_ant)
if not os.path.isdir(folder_path):
    Path(folder_path).mkdir(exist_ok=True)

result_filepath = os.path.join(folder_path, 'egs_set_2_seed_100.pkl') 

ftpl = []
nonftpl = []
MODEL_FILEPATH = '/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/trained_models/N={},M={},L={}/gnn_ftpl_iter_17'.format(N,M,max_ant)
# MODEL_FILEPATH = '/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/trained_models/N=8,M=3,L=5/gnn_ftpl_iter_30'.format(N,M,max_ant)
# MODEL_FILEPATH = '/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/trained_models/N=12,M=6,L=8/gnn_ftpl_iter_11'.format(N,M,max_ant)
MODEL_FOLDERPATH ='/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/trained_models/N={},M={},L={}'.format(N,M,max_ant)

num_egs = 30
np.random.seed(seed = 100)
num_nodes_eval = 0
time_total = 0
obj_bb = 0
obj_omar = 0
obj_opt = 0

t1 = time.time()

instances = np.random.randn(num_egs, 2, N, M)

######### ORACLE method #####################

t1 = time.time()
arguments_oracle = list(zip(list(instances), [max_ant]*num_egs))


out_oracle =[]
for i in range(num_egs):
    output = solve_bb_pool(arguments_oracle[i])
    out_oracle.append(output)
    

# with Pool(num_egs) as p:
#     out_oracle = p.map(solve_bb_pool, arguments_oracle)
#     print('pool ended')
oracle_time = time.time() - t1

optimal_solution_list = [out_oracle[i][0] for i in range(len(out_oracle))]
optimal_objective_list = [out_oracle[i][1] for i in range(len(out_oracle))]
oracle_timesteps = [out_oracle[i][2] for i in range(len(out_oracle))]
oracle_time = np.mean([out_oracle[i][3] for i in range(len(out_oracle))])

data = (instances, optimal_solution_list, optimal_objective_list, oracle_time, oracle_timesteps)
with open(result_filepath, 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)    

########### LOAD data ################################################

with open(result_filepath, 'rb') as handle:
    data = pickle.load(handle)
    instances, optimal_solution_list, optimal_objective_list, oracle_time, oracle_timesteps = data

########## OMAR'S Method #######################################
# mask_omar = []
# obj_omar = []
# omar_time = 0
# use_omar = []
# for i in range(num_egs):
#     H = instances[i,0,:,:] + 1j*instances[i,1,:,:]
#     obj_bb += optimal_objective_list[i]
#     t1 = time.time()
#     try:
#         obj, mask = as_omar(H, max_ant=max_ant)
#         use_omar.append(True)
#     except:
#         obj = None
#         mask = np.zeros(N)
#         use_omar.append(False)
        
#     omar_time += time.time() - t1
#     obj_omar.append(obj)
#     mask_omar.append(mask)
# omar_time = omar_time/num_egs
# print(omar_time)


########### GNN MODEL ###################################

# # skip heuristics:
# mask_omar = [np.zeros(N) for _ in range(num_egs)]
# use_omar = [False for _ in range(num_egs)]



# result = []
# for i in range(4,20):
#     MODEL_FILEPATH = os.path.join(MODEL_FOLDERPATH, 'gnn_ftpl3_iter_{}'.format(i))
#     t1 = time.time()
#     arguments_ml = list(zip(list(instances), optimal_solution_list, optimal_objective_list, range(num_egs), [MODEL_FILEPATH]*num_egs, [max_ant]*num_egs, mask_omar, use_omar))
    
#     out_ml = []
# #     for i in range(num_egs):
# #         print('eg {}'.format(i))
# #         output = solve_ml_pool(arguments_ml[i])
# #         out_ml.append(output)

#     with Pool(num_egs) as p:
#         out_ml = p.map(solve_ml_pool, arguments_ml)
    
#     avg_ml_ogap = np.mean(np.array([(out_ml[i][1]>-1)*out_ml[i][1] for i in range(len(out_ml))]))
#     solution_rate = np.sum(np.array([(out_ml[i][1]>-1) for i in range(len(out_ml))]))
#     avg_ml_steps = np.mean(np.array([out_ml[i][0] for i in range(len(out_ml))]))
#     ml_time = np.mean(np.array([out_ml[i][2] for i in range(len(out_ml))]))
#     result.append(('model: {}'.format(i), avg_ml_ogap, solution_rate, avg_ml_steps, ml_time))
    

timestep 0 inf 0.6028728127466061
timestep 1 inf 0.6028726093281387
timestep 2 inf 0.6028729811554702
timestep 3 inf 0.6028728749880388
timestep 4 7.591410666082475 0.678979648025724
timestep 5 7.591410666082475 0.7214990833787486
timestep 6 6.794000462535628 0.7345476800638807
timestep 7 6.794000462535628 0.7345475768708778
timestep 8 6.794000462535628 0.7444611224425347
timestep 9 6.794000462535628 0.7444611797605708
timestep 10 6.794000462535628 0.7444612681616835
timestep 11 6.794000462535628 0.7444610498287962
timestep 12 6.794000462535628 0.7501844570989411
timestep 13 6.794000462535628 0.7501842626674862
timestep 14 6.794000462535628 0.7501842463680878
timestep 15 6.794000462535628 0.7797865719833744
timestep 16 6.794000462535628 0.8009125939680773
timestep 17 6.794000462535628 0.8009126725811588
timestep 18 6.794000462535628 0.8009126600506563
timestep 19 6.794000462535628 0.8189024637057296
timestep 20 6.794000462535628 0.8262859230326501
timestep 21 6.794000462535628 0.826286

timestep 169 4.80367801370609 1.8829841975654535
timestep 170 4.80367801370609 1.8927099225663537
timestep 171 4.80367801370609 1.8951862073627652
timestep 172 4.80367801370609 1.8951861607897704
timestep 173 4.80367801370609 1.9107657086606864
timestep 174 4.80367801370609 1.9332752676384464
timestep 175 4.80367801370609 1.9389048636701733
timestep 176 4.80367801370609 1.9524605959835408
timestep 177 4.80367801370609 1.9524600653620687
timestep 178 4.80367801370609 1.964305746983818
timestep 179 4.80367801370609 1.9718642615379334
timestep 180 4.80367801370609 1.9786019639674293
timestep 181 4.80367801370609 2.0228636892316922
timestep 182 4.80367801370609 2.0228641686618087
timestep 183 4.80367801370609 2.0228637900067965
timestep 184 4.80367801370609 2.023751239187492
timestep 185 4.80367801370609 2.0295927804615395
timestep 186 4.80367801370609 2.029593551406255
timestep 187 4.80367801370609 2.035801052267309
timestep 188 4.80367801370609 2.0857161608746315
timestep 189 4.803678013

timestep 32 4.161599975703371 0.5701163475939082
timestep 33 4.161599975703371 0.5701163572693202
timestep 34 4.161599975703371 0.6135090937520491
timestep 35 4.161599975703371 0.6146546484136367
timestep 36 4.161599975703371 0.6146543440498107
timestep 37 4.161599975703371 0.6270145493210404
timestep 38 4.161599975703371 0.6270142264043007
timestep 39 4.161599975703371 0.634763993060654
timestep 40 4.161599975703371 0.6370998605701467
timestep 41 4.161599975703371 0.637912598383071
timestep 42 4.161599975703371 0.6379129319652062
timestep 43 4.161599975703371 0.6436419354371966
timestep 44 4.161599975703371 0.6470799006396002
timestep 45 4.161599975703371 0.6540230830882277
timestep 46 4.161599975703371 0.6540229217225367
timestep 47 4.161599975703371 0.6540229228669893
timestep 48 4.161599975703371 0.6540230227878685
timestep 49 4.161599975703371 0.657725775305709
timestep 50 4.161599975703371 0.6577255412320835
timestep 51 4.161599975703371 0.6600110152252906
timestep 52 4.161599975

timestep 198 4.161599975703371 1.5348757649451752
timestep 199 4.161599975703371 1.534875567546189
timestep 200 4.161599975703371 1.5348757850442758
timestep 201 4.161599975703371 1.537131082410531
timestep 202 4.161599975703371 1.537650172041326
timestep 203 4.161599975703371 1.5379994904113476
timestep 204 4.161599975703371 1.5380001390572156
timestep 205 4.161599975703371 1.5379997457754686
timestep 206 4.161599975703371 1.537999951615363
timestep 207 4.161599975703371 1.5583276955201004
timestep 208 4.161599975703371 1.5583277643872615
timestep 209 4.161599975703371 1.5603529557867868
timestep 210 4.161599975703371 1.5622082520006866
timestep 211 4.161599975703371 1.5634556729271765
timestep 212 4.161599975703371 1.5691806442617082
timestep 213 4.0644170447015835 1.5716307932095073
timestep 214 4.0644170447015835 1.5716306804675209
timestep 215 4.0644170447015835 1.5850895998699575
timestep 216 4.0644170447015835 1.5867250593376105
timestep 217 4.0644170447015835 1.5930081516334018

timestep 359 3.595102880076599 3.587848261269798
timestep 360 3.595102880076599 3.587847603556965
infeasible solution
ended
timestep 0 inf 0.37247239954907524
timestep 1 inf 0.37247213605563734
timestep 2 inf 0.3724722631632736
timestep 3 inf 0.37247213986747846
timestep 4 inf 0.3963392711895878
timestep 5 inf 0.3963390163171306
timestep 6 inf 0.3963392543865726
timestep 7 5.576683816317827 0.3979707472968473
timestep 8 5.576683816317827 0.39797078431672694
timestep 9 5.576683816317827 0.41585443992690846
timestep 10 5.576683816317827 0.4252910445944598
timestep 11 3.126592587680175 0.4442745926263593
timestep 12 3.126592587680175 0.4442744537630412
timestep 13 3.126592587680175 0.44427442207446827
timestep 14 3.126592587680175 0.44427443896900765
timestep 15 3.126592587680175 0.46467674163638334
timestep 16 3.126592587680175 0.4663609105862413
timestep 17 3.126592587680175 0.46636098294339434
timestep 18 3.126592587680175 0.4709404441340361
timestep 19 3.126592587680175 0.470940252883

...

timestep 40 4.324476482648396 0.7430121693290905
timestep 41 4.324476482648396 0.7506670925575966
timestep 42 4.324476482648396 0.7506673059937601
timestep 43 4.324476482648396 0.7684352714328095
timestep 44 4.324476482648396 0.7764324140246762
timestep 45 4.324476482648396 0.7838847880475586
timestep 46 4.324476482648396 0.7838848463405343
timestep 47 4.324476482648396 0.7864235101604127
timestep 48 4.324476482648396 0.7864230465896552
timestep 49 4.324476482648396 0.7864233374588661
timestep 50 4.324476482648396 0.7916388850924662
timestep 51 4.195915640649826 0.7966280051731406
timestep 52 4.195915640649826 0.8121148372173933
timestep 53 4.195915640649826 0.8336798264418965
timestep 54 4.195915640649826 0.83367973799341
timestep 55 4.195915640649826 0.8336795905267094
timestep 56 4.195915640649826 0.8336794437829657
timestep 57 4.195915640649826 0.8503996154644049
timestep 58 4.195915640649826 0.8503994997431149
timestep 59 4.195915640649826 0.851466481689264
timestep 60 4.195915640

timestep 206 3.61377771785912 1.7735791399277392
timestep 207 3.61377771785912 1.8023945986918066
timestep 208 3.61377771785912 1.8023947126563549
timestep 209 3.61377771785912 1.8101289369948395
timestep 210 3.61377771785912 1.8101290499881446
timestep 211 3.61377771785912 1.8101287904751673
timestep 212 3.61377771785912 1.815611558498976
timestep 213 3.61377771785912 1.8171546391961513
timestep 214 3.61377771785912 1.8191419481370503
timestep 215 3.61377771785912 1.8191419518314642
timestep 216 3.61377771785912 1.819934182090416
timestep 217 3.61377771785912 1.8199338135098808
timestep 218 3.61377771785912 1.823115383172873
timestep 219 3.61377771785912 1.8415318759462307
timestep 220 3.61377771785912 1.864517490476802
timestep 221 3.61377771785912 1.8744302758270839
timestep 222 3.61377771785912 1.8744310156872275
timestep 223 3.61377771785912 1.9423802915763029
timestep 224 3.61377771785912 1.9450221910365566
timestep 225 3.61377771785912 1.9487282689297565
timestep 226 3.613777717

timestep 35 6.405513841838532 0.9023476502716551
timestep 36 6.405513841838532 0.9028657049916599
timestep 37 6.405513841838532 0.9063570795097159
timestep 38 6.405513841838532 0.9063565601326536
timestep 39 6.405513841838532 0.9063567825409612
timestep 40 6.405513841838532 0.9104786841414156
timestep 41 6.405513841838532 0.9104785935637447
timestep 42 6.405513841838532 0.910478711266949
timestep 43 6.405513841838532 0.9106720261331059
timestep 44 6.405513841838532 0.9106716840291765
timestep 45 5.605676755440215 0.9260219114714392
timestep 46 5.605676755440215 0.9985624216281072
timestep 47 5.605676755440215 0.9985628830077421
timestep 48 5.605676755440215 1.0034694582735388
timestep 49 5.605676755440215 1.003469495223766
timestep 50 5.605676755440215 1.0105553129270939
timestep 51 5.605676755440215 1.010554912312756
timestep 52 5.605676755440215 1.0116171845754158
timestep 53 5.605676755440215 1.0328705649737382
timestep 54 5.605676755440215 1.0328706263026866
timestep 55 5.605676755

timestep 202 5.605676755440215 2.155063362852458
timestep 203 5.605676755440215 2.173313567244213
timestep 204 5.605676755440215 2.18164848478063
timestep 205 5.605676755440215 2.1852746490244206
timestep 206 5.605676755440215 2.1852748651208493
timestep 207 5.605676755440215 2.1925649799804265
timestep 208 5.605676755440215 2.1937349457272806
timestep 209 5.605676755440215 2.193734225797378
timestep 210 5.605676755440215 2.2162824233786225
timestep 211 5.605676755440215 2.216282045496687
timestep 212 5.605676755440215 2.225036890827652
timestep 213 5.605676755440215 2.2264066453014344
timestep 214 5.605676755440215 2.2264066266984073
timestep 215 5.605676755440215 2.226406888329459
timestep 216 5.605676755440215 2.2446729882689356
timestep 217 5.605676755440215 2.2511802357159363
timestep 218 5.605676755440215 2.257081209221698
timestep 219 5.605676755440215 2.2570812786123238
timestep 220 5.605676755440215 2.257081422095976
timestep 221 5.605676755440215 2.2613983372002573
timestep 2

timestep 364 5.605676755440215 4.4489521145749285
timestep 365 5.605676755440215 4.448951517653626
infeasible solution
timestep 366 5.605676755440215 4.509266965046673
timestep 367 5.605676755440215 4.695894517321112
timestep 368 5.605676755440215 4.741646582734898
infeasible solution
timestep 369 5.605676755440215 4.77246224151696
infeasible solution
timestep 370 5.605676755440215 4.772462180623362
timestep 371 5.605676755440215 4.803304788858836
timestep 372 5.605676755440215 4.8690616811665635
timestep 373 5.605676755440215 4.869060639205592
timestep 374 5.605676755440215 4.869061787158359
infeasible solution
timestep 375 5.605676755440215 4.882099328222634
timestep 376 5.605676755440215 4.8933351855604785
infeasible solution
timestep 377 5.605676755440215 4.8933356707622835
timestep 378 5.605676755440215 4.97649942956797
timestep 379 5.605676755440215 5.103783907707644
timestep 380 5.605676755440215 5.103783449810196
timestep 381 5.605676755440215 5.112893052130164
infeasible solut

In [17]:
result

[('model: 4', 3.227112590493447, 30, 20.733333333333334, 99.76462252140045),
 ('model: 5', 5.6137565560736675, 30, 15.933333333333334, 73.38656240304311),
 ('model: 6', 4.368127494826608, 30, 17.2, 82.99203956921896),
 ('model: 7', 3.5877775071791453, 30, 21.066666666666666, 102.8183926820755),
 ('model: 8', 4.368127494826608, 30, 19.0, 95.32836592197418),
 ('model: 9', 7.177321154147128, 30, 13.766666666666667, 68.29467584292094),
 ('model: 10', 6.8713102117408145, 30, 16.833333333333332, 84.31744290192923),
 ('model: 11', 2.593051546673186, 30, 19.966666666666665, 97.0886871099472),
 ('model: 12', 6.8713102117408145, 30, 16.933333333333334, 83.77808713118235),
 ('model: 13', 16.692499993609353, 30, 18.166666666666668, 93.02174173990885),
 ('model: 14', 4.192406760437419, 30, 20.566666666666666, 105.67746328512827),
 ('model: 15', 2.593051546673186, 30, 17.833333333333332, 83.92939085165659),
 ('model: 16', 4.148225892140609, 30, 18.6, 85.76366446018218),
 ('model: 17', 2.065840355229

In [2]:
result

[('model: 4', 3.227112590493447, 30, 20.733333333333334, 156.73719321091968)]

In [11]:
result

[('model: 4', 2.0658403552292293, 30, 21.233333333333334, 85.00518458684286)]

In [12]:
oracle_time/85.005

2.5603213622598413

In [18]:
56/17.83

3.1407739764441955

In [40]:
oracle_time/236

3.1969151071909456

In [26]:
ogap = [(out_ml[i][1]>-1000)*out_ml[i][1] for i in range(len(out_ml))]

In [35]:
ogap[0] = 77

In [29]:
obj_omar = [3.767733304451285,
 None,
 None,
 None,
 3.894313287867485,
 None,
 2.807588738119765,
 None,
 None,
 3.2634901523605344,
 4.50767154557565,
 4.153571111035874,
 3.801474133683451,
 3.765459336092195,
 2.504905077133235,
 None,
 3.0036147698458224,
 3.044519372122908,
 4.041942169492759,
 2.962729048320993,
 2.0339742795292177,
 None,
 2.628754213643843,
 2.803554689922972,
 3.7156776714065662,
 3.699832443501566,
 3.7096730221417222,
 None,
 None,
 None]

In [2]:
result

[('model: 3', 4.63190968212591, 29, 51.63333333333333, 233.8756853421529)]

## Proposed: baseline solved cases only

In [13]:
ogap_ml = 0
ogap = [(out_ml[i][1]>-1000)*out_ml[i][1] for i in range(len(out_ml))]
solved_instances = 0
for i in range(len(obj_omar)):
    if obj_omar[i] is not None:
        ogap_ml += ogap[i]
        solved_instances += 1

In [14]:
solved_instances

13

In [15]:
ogap_ml/solved_instances

0.7665759570472801

In [19]:
ogap_ml/solved_instances

3.374235703486729

In [18]:
solution_rate/num_egs

0.7333333333333333

In [14]:
avg_ml_steps

49.4

In [15]:
ml_time

242.25424818197885

In [3]:
omar_obj = obj_omar
ogap = 0
solved_instances = 0
for i in range(len(omar_obj)):
    if omar_obj[i] is not None:
        gap = (omar_obj[i] - optimal_objective_list[i])/optimal_objective_list[i] *100
        print(gap)
        ogap += gap
        solved_instances += 1
    else:
        print('None')
print(ogap/solved_instances)


None
-4.129056088171869e-14
7.892116835279434
None
None
None
None
None
None
6.006510736822515
3.015723143368322e-14
None
0.0
None
None
14.072830671358085
0.0
None
8.742593006427377
0.0
None
None
None
7.945640400158766
0.0
0.0
45.481591549619665
None
None
None
6.933944861512757


In [4]:
solved_instances/num_egs

0.43333333333333335

In [1]:
import numpy as np

a = np.random.randn(5)

In [7]:

b = [(a[i]>0)*a[i] for i in range(5)]

In [8]:
b

[0.19838856229286603, 0.14537281803798, 1.5687498144513823, -0.0, -0.0]

In [9]:
a

array([ 0.19838856,  0.14537282,  1.56874981, -1.23274031, -0.65205485])

In [4]:
out_ml

[(21, -100.0, 216.95999026298523, 0.16279069767441862),
 (77, 15.75746548912954, 462.85954570770264, 0.06779661016949153),
 (78, 0.0, 463.6005153656006, 0.0847457627118644),
 (61, 0.0, 458.06463956832886, 0.07476635514018691),
 (40, 42.35309371027526, 355.954815864563, 0.07142857142857142),
 (25, -100.0, 252.4719569683075, 0.1568627450980392),
 (60, 52.132099698287504, 453.6423716545105, 0.0594059405940594),
 (30, -100.0, 298.6989555358887, 0.08196721311475409),
 (35, -100.0, 341.4387152194977, 0.11267605633802817),
 (63, 34.82650888337098, 449.44190526008606, 0.05319148936170213),
 (70, 25.110955287464897, 458.9494686126709, 0.07692307692307693),
 (21, -100.0, 219.5476553440094, 0.11627906976744186),
 (29, -100.0, 298.31939578056335, 0.13559322033898305),
 (48, -100.0, 424.04892015457153, 0.07216494845360824),
 (29, 0.0, 258.27710723876953, 0.2127659574468085),
 (52, 1.5451861737475518, 419.78126311302185, 0.08139534883720931),
 (62, 45.4170633036014, 446.2941017150879, 0.097826086956

In [7]:
[out_ml[i][1] for i in range(len(out_ml))]

[-100.0,
 1042.1954585039352,
 0.0,
 0.0,
 7.680004624323597,
 -100.0,
 420.33116215453214,
 -100.0,
 -100.0,
 59.96839470677195,
 25.110955287464897,
 165.3409165564066,
 -100.0,
 -100.0,
 41.67005035746222,
 1.6093020953382209,
 91.55603638000123,
 730.8125814114214,
 53.53164428978343,
 297.20664459410875,
 20.483362398107754,
 286.9369284184948,
 61.00091255801037,
 0.6254527167696247,
 428.9813151571964,
 70.2357085702821,
 0.0,
 99.95579292293255,
 23.985228060921013,
 8.623371382064157]

In [3]:
optimal_solution_list

[array([-0., -0., -0.,  1., -0., -0.,  1., -0., -0.,  1., -0.,  1.]),
 array([ 0.,  0., -0.,  0., -0., -0.,  1., -0.,  1., -0.,  1.,  1.]),
 array([-0., -0., -0., -0., -0., -0., -0.,  1., -0.,  1.,  1.,  1.]),
 array([ 1.,  1., -0., -0., -0.,  0.,  1.,  0.,  1., -0.,  0.,  0.]),
 array([ 0., -0.,  1., -0.,  1., -0.,  0., -0.,  1.,  0.,  1., -0.]),
 array([-0.,  1.,  1.,  1., -0., -0., -0., -0., -0.,  1., -0., -0.]),
 array([1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0.]),
 array([ 1.,  1.,  0., -0., -0.,  0.,  0.,  0.,  0.,  1., -0.,  1.]),
 array([ 1., -0.,  1.,  0.,  0.,  0., -0.,  1., -0., -0.,  1., -0.]),
 array([ 1., -0., -0.,  0., -0.,  1.,  1.,  1.,  0., -0., -0., -0.]),
 array([-0., -0., -0., -0.,  1., -0.,  1.,  0.,  1., -0.,  1., -0.]),
 array([ 1., -0.,  0., -0.,  0., -0.,  1.,  0., -0.,  0.,  1.,  1.]),
 array([ 1.,  1., -0., -0.,  1.,  1., -0., -0., -0.,  0., -0., -0.]),
 array([-0., -0., -0.,  0.,  1.,  1.,  0.,  0., -0.,  1.,  1.,  0.]),
 array([ 0., -0., -0., -0., -0.,

In [4]:
optimal_objective_list

[3.767733304451285,
 3.595102880076599,
 2.680568611484319,
 5.491072831697681,
 3.894313287867485,
 5.121136719233905,
 2.807588738119765,
 5.371181050620627,
 3.5569921077779254,
 3.2634901523605344,
 4.50767154557565,
 4.153571111035874,
 3.7361465291274936,
 2.5509575447393003,
 2.504905077133235,
 4.0065371295493435,
 2.6459662393207677,
 2.7678068220727696,
 4.041942169492759,
 2.209942488720159,
 2.0339742795292177,
 3.5976715276184077,
 2.287898597334443,
 2.78612877182688,
 3.0874998478603852,
 3.2073613380006,
 3.709673022141723,
 4.34540249808348,
 3.61377771785912,
 5.605676755440215]

In [14]:
omar_obj = obj_omar
ogap = 0
solved_instances = 0
for i in range(len(omar_obj)):
    if omar_obj[i] is not None:
        gap = (omar_obj[i] - optimal_objective_list[i])/optimal_objective_list[i] *100
        print(gap)
        ogap += gap
        solved_instances += 1
    else:
        print('None')
print(ogap/solved_instances)


0.0
None
None
None
0.0
None
0.0
None
None
0.0
0.0
0.0
1.748528973546808
47.609643439872016
0.0
None
13.51674580008492
9.997538406344134
0.0
34.063626698122555
0.0
None
14.898195956172172
0.6254527167696565
20.345841441304156
15.354400505680527
-2.3942229258452245e-14
None
None
None
8.324209154626153


In [13]:
[out_ml[i][1] for i in range(len(out_ml))]

[-100.0,
 1042.1954585039352,
 0.0,
 0.0,
 7.680004624323597,
 -100.0,
 420.33116215453214,
 -100.0,
 -100.0,
 59.96839470677195,
 25.110955287464897,
 165.3409165564066,
 -100.0,
 -100.0,
 41.67005035746222,
 1.6093020953382209,
 91.55603638000123,
 730.8125814114214,
 53.53164428978343,
 297.20664459410875,
 20.483362398107754,
 286.9369284184948,
 61.00091255801037,
 0.6254527167696247,
 428.9813151571964,
 70.2357085702821,
 0.0,
 99.95579292293255,
 23.985228060921013,
 8.623371382064157]

In [4]:
print('Omar Ogap', (obj_omar-obj_bb)/obj_bb*100)


Omar Ogap 1.1208451189963875


In [5]:
omar_time

0.9480806827545166

In [2]:
1010.29/5.16

195.79263565891472

In [ ]:
instance result 29 11 0.0 6.056657075881958 7 0.45573338907549404 0.45573338907549404
Model ogap: 0.5721763222161272, time speedup: 4.672709040459938
BB time: 19.966274698575337, ML time: 4.272954837481181 
        
Omar 

## GNN Model

In [1]:
import numpy as np
import itertools
from beamforming import solve_beamforming_with_selected_antennas
from multiprocessing import Pool
from as_omar import as_omar
# from as_bb import solve_bb, DefaultBranchingPolicy
# from as_bb import ASBBenv as Environment

from as_bb_test import solve_bb, DefaultBranchingPolicy
from as_bb_test import ASBBenv as Environment
import os
import pickle
import time
from pathlib import Path
from antenna_selection.observation import Observation
from antenna_selection.model_tester import *

def solve_bb_pool(arguments):
    instance, max_ant = arguments
    return solve_bb(instance, max_ant=max_ant)


# def collect_data_instance(arguments):
    
#     instance, w_optimal, optimal_objective, file_count, policy_filepath, max_ant, mask_omar = arguments

#     env = Environment(observation_function=Observation, epsilon=0.002)
#     env.set_node_select_policy(node_select_policy_path=policy_filepath, policy_type='gnn')
# #     env.set_heuristic_solutions(mask_omar)

#     env.reset(instance, max_ant=max_ant,  oracle_opt=w_optimal)

#     branching_policy = DefaultBranchingPolicy()
#     t1 = time.time()

#     timestep = 0
#     done = False
#     time_taken = 0
#     sum_label = 0
#     node_counter = 0

#     while timestep < 1000 and len(env.nodes)>0 and not done: 
#         print('timestep {}'.format(timestep))
# #         print('timestep {} U {}, L {}'.format(timestep, env.global_U, env.global_L))

#         t2 = time.time()
#         env.fathom_nodes()
#         if len(env.nodes) == 0:
#             break
#         node_id, node_feats, label = env.select_node()
#         # print('observation instance', isinstance(node_feats, Observation))

#         if len(env.nodes) == 0:
#             break
#         time_taken += time.time()-t1
#         sum_label += label  
#         node_counter += 1
#         t1 = time.time()

#         print("Selection Time {}".format(time.time()-t2))
#         t2 = time.time()

#         prune_node = env.prune(node_feats)
#         # prune_node = False
        
#         print("Prune Decision Time {}".format(time.time()-t2))
#         t2 = time.time()

#         if prune_node:
#             env.delete_node(node_id)
#             continue
#         else:
#             branching_var = branching_policy.select_variable(node_feats, env.action_set_indices)
#             # try:
#             #     done = env.push_children(branching_var, node_id)
#             # except:
#             #     print("exception occured")
#             #     break
#             done = env.push_children(branching_var, node_id)
        
#         print("Push children Time {}".format(time.time()-t2))
        
#         timestep = timestep+1

#     ml = np.linalg.norm(env.W_incumbent, 'fro')**2
#     ogap = ((ml - optimal_objective)/optimal_objective)*100
#     time_taken += time.time() - t1
#     print('instance result',file_count, timestep, ogap, time_taken, sum_label, optimal_objective, ml)
#     return timestep, ogap, time_taken, sum_label/node_counter

N = 12
M = 8
max_ant = 4
folder_path = '/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/validation_set/N={},M={},L={}_ftpl'.format(N,M,max_ant)
if not os.path.isdir(folder_path):
    Path(folder_path).mkdir(exist_ok=True)

result_filepath = os.path.join(folder_path, 'egs_set_2_seed_100.pkl') 

ftpl = []
nonftpl = []
# MODEL_FILEPATH = '/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/trained_models/N={},M={},L={}/gnn1_iter_9'.format(N,M,max_ant)
# MODEL_FILEPATH = '/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/trained_models/N=8,M=3,L=5/gnn_ftpl_iter_28'.format(N,M,max_ant)
# MODEL_FILEPATH = '/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/trained_models/N=12,M=6,L=8/gnn_ftpl_iter_10'.format(N,M,max_ant)
MODEL_FILEPATH = '/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/trained_models/N=12,M=8,L=4/gnn_ftpl3_iter_6'.format(N,M,max_ant)

num_egs = 30
np.random.seed(seed = 100)
num_nodes_eval = 0
time_total = 0
obj_bb = 0
obj_omar = 0
obj_opt = 0
import time
t1 = time.time()

instances = np.random.randn(num_egs, 2, N, M)

########### LOAD data ################################################

with open(result_filepath, 'rb') as handle:
    data = pickle.load(handle)
    instances, optimal_solution_list, optimal_objective_list, oracle_time, oracle_timesteps = data


mask_omar = []
obj_omar = []
omar_time = 0
use_omar = []
for i in range(num_egs):
    H = instances[i,0,:,:] + 1j*instances[i,1,:,:]
    obj_bb += optimal_objective_list[i]
    t1 = time.time()
    try:
        obj, mask = as_omar(H, max_ant=max_ant)
        use_omar.append(True)
    except:
        obj = None
        mask = np.zeros(N)
        use_omar.append(False)
        
    omar_time += time.time() - t1
    obj_omar.append(obj)
    mask_omar.append(mask)
omar_time = omar_time/num_egs
print(omar_time)


sparse iteration  0
sparse iteration  1
sparse iteration  2
sparse iteration  3
sparse iteration  4
sparse iteration  5
sparse iteration  6
exiting here
num selected antennas 4
3.767733304451285
Before lambda iteration: 4
After lambda iteration: 4
sparse iteration  0
sparse iteration  1
sparse iteration  2
sparse iteration  3
sparse iteration  4
sparse iteration  5
sparse iteration  6
sparse iteration  7
sparse iteration  8
sparse iteration  9
sparse iteration  10
sparse iteration  11
sparse iteration  12
sparse iteration  13
sparse iteration  14
sparse iteration  15
sparse iteration  16
sparse iteration  17
sparse iteration  18
sparse iteration  19
sparse iteration  20
sparse iteration  21
sparse iteration  22
sparse iteration  23
sparse iteration  24
sparse iteration  25
sparse iteration  26
sparse iteration  27
sparse iteration  28
sparse iteration  29
sparse iteration  0
sparse iteration  1
sparse iteration  2
sparse iteration  3
sparse iteration  4
sparse iteration  5
sparse itera

sparse iteration  10
exiting here
num selected antennas 4
4.041942169492759
Before lambda iteration: 4
After lambda iteration: 4
sparse iteration  0
sparse iteration  1
sparse iteration  2
sparse iteration  3
sparse iteration  4
exiting here
num selected antennas 4
2.962729048320993
Before lambda iteration: 4
After lambda iteration: 4
sparse iteration  0
sparse iteration  1
sparse iteration  2
exiting here
num selected antennas 4
2.0339742795292177
Before lambda iteration: 4
After lambda iteration: 4
sparse iteration  0
sparse iteration  1
sparse iteration  2
sparse iteration  3
sparse iteration  4
sparse iteration  5
sparse iteration  6
sparse iteration  7
sparse iteration  8
sparse iteration  9
sparse iteration  10
sparse iteration  11
sparse iteration  12
sparse iteration  13
sparse iteration  14
sparse iteration  15
sparse iteration  16
sparse iteration  17
sparse iteration  18
sparse iteration  19
sparse iteration  20
sparse iteration  21
sparse iteration  22
sparse iteration  23


In [48]:
np.mean(oracle_timesteps)

336.3333333333333

In [49]:
oracle_time

140.90131459236144

In [ ]:
ogap_ml = 0
ogap = [(out_ml[i][1]>-1000)*out_ml[i][1] for i in range(len(out_ml))]
tsteps_ml = 0
solved_instances = 0
for i in range(len(obj_omar)):
    if obj_omar[i] is not None:
        ogap_ml += ogap[i]
        solved_instances += 1


In [7]:
ogap_omar = []
for i in range(len(obj_omar)):
    if obj_omar[i] is not None:
        ogap_omar.append(obj_omar[i])

In [8]:
np.mean(ogap_omar)

3.374235703486729

In [2]:
use_mask = []
for i in range(len(obj_omar)):
    if obj_omar[i] is not None:
        use_mask.append(True)
    else:
        use_mask.append(False)

In [30]:
oracle_time/omar_time

18.441974282366502

In [ ]:
oracle_time

In [10]:
len(ogap_omar)

19

In [11]:
obj_omar

[3.767733304451285,
 None,
 None,
 None,
 3.894313287867485,
 None,
 2.807588738119765,
 None,
 None,
 3.2634901523605344,
 4.50767154557565,
 4.153571111035874,
 3.801474133683451,
 3.765459336092195,
 2.504905077133235,
 None,
 3.0036147698458224,
 3.044519372122908,
 4.041942169492759,
 2.962729048320993,
 2.0339742795292177,
 None,
 2.628754213643843,
 2.803554689922972,
 3.7156776714065662,
 3.699832443501566,
 3.7096730221417222,
 None,
 None,
 None]

In [24]:
print((obj_omar-obj_bb)/obj_bb)

0.0236632545782786


In [3]:
# mask_omar = [np.zeros(N) for _ in range(num_egs)]

t1 = time.time()

offset = 0
arguments_ml = list(zip(list(instances)[offset:], optimal_solution_list[offset:], optimal_objective_list[offset:], range(num_egs), [MODEL_FILEPATH]*num_egs, [max_ant]*num_egs, mask_omar, use_mask))

out_ml =[]
for i in range(num_egs):
    print('example ', i)
    if obj_omar[i] is not None:
        output = solve_ml_pool(arguments_ml[i])
        out_ml.append(output)
    

avg_ml_ogap = np.mean(np.array([out_ml[i][1] for i in range(len(out_ml))]))
avg_ml_steps = np.mean(np.array([out_ml[i][0] for i in range(len(out_ml))]))
ml_time = np.mean(np.array([out_ml[i][2] for i in range(len(out_ml))]))

# time profiling
misc_time =           np.mean(np.array([out_ml[i][4][0] for i in range(len(out_ml))]))
classification_time = np.mean(np.array([out_ml[i][4][1] for i in range(len(out_ml))]))
branching_time =      np.mean(np.array([out_ml[i][4][2] for i in range(len(out_ml))]))
classified_nodes =    np.mean(np.array([out_ml[i][4][3] for i in range(len(out_ml))]))
branched_nodes =      np.mean(np.array([out_ml[i][4][4] for i in range(len(out_ml))]))

print('Model ogap: {}, time speedup: {}'.format(avg_ml_ogap, oracle_time/ml_time))
print('BB time: {}, ML time: {} '.format(oracle_time, ml_time))

example  0
policy path /scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/trained_models/N=12,M=8,L=4/gnn_ftpl3_iter_6


/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/as_bb_test.py:436: RuntimeWarning: invalid value encountered in double_scalars
  if (self.global_U - self.global_L)/abs(self.global_U) < self.epsilon:


timestep 0


/scratch/sagar/Projects/combopt/venv/lib/python3.6/site-packages/cvxpy/reductions/solvers/solving_chain.py:167: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)


timestep 1
timestep 2
timestep 3
timestep 4
timestep 5
timestep 6
timestep 7
timestep 8
timestep 9
timestep 10
timestep 11
timestep 11
timestep 12
timestep 12
timestep 13
timestep 14
timestep 15
timestep 15
timestep 16
timestep 16
timestep 17
timestep 18
timestep 18
timestep 18
timestep 19
timestep 20
timestep 21
timestep 22
timestep 22
timestep 22
timestep 23
timestep 24
timestep 24
timestep 24
timestep 24
timestep 25
timestep 25
timestep 25
timestep 25
timestep 26
timestep 26
timestep 26
timestep 27
timestep 27
instance result 28 0.0 0.3684072494506836 11 3.767733304451285 3.767733304451285
example  1
example  2
example  3
example  4
policy path /scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/trained_models/N=12,M=8,L=4/gnn_ftpl3_iter_6
timestep 0
timestep 1
timestep 2
timestep 3
timestep 3
timestep 4
timestep 5
timestep 5
timestep 6
timestep 6
timestep 7
timestep 8
timestep 9
timestep 9
timestep 10
timestep 10
timestep 11
timestep 11
timestep 12
timestep 12
tim

timestep 0
timestep 1
timestep 2
timestep 3
timestep 3
timestep 4
timestep 5
timestep 6
timestep 6
timestep 7
timestep 8
timestep 8
timestep 9
timestep 10
timestep 11
timestep 11
timestep 12
timestep 12
timestep 13
timestep 14
timestep 14
timestep 15
timestep 15
timestep 16
timestep 16
timestep 17
timestep 18
timestep 18
timestep 19
timestep 20
timestep 21
timestep 21
timestep 22
timestep 22
timestep 23
timestep 24
timestep 24
timestep 25
timestep 25
timestep 26
timestep 27
timestep 27
timestep 28
timestep 29
timestep 29
timestep 30
timestep 30
timestep 31
timestep 31
timestep 32
timestep 33
timestep 33
timestep 34
timestep 35
timestep 36
timestep 36
timestep 37
timestep 37
timestep 38
timestep 38
timestep 38
timestep 39
timestep 40
timestep 41
timestep 41
timestep 41
timestep 41
timestep 41
timestep 41
timestep 42
timestep 43
timestep 43
timestep 43
timestep 43
timestep 43
timestep 43
timestep 43
timestep 43
timestep 44
timestep 45
timestep 45
timestep 45
timestep 46
timestep 47
times

timestep 20
timestep 21
timestep 22
timestep 22
timestep 23
timestep 23
timestep 24
timestep 25
timestep 26
timestep 27
timestep 28
timestep 29
timestep 30
timestep 31
timestep 32
timestep 33
timestep 34
timestep 35
timestep 36
timestep 36
timestep 37
timestep 38
timestep 39
timestep 39
timestep 40
timestep 40
timestep 40
timestep 41
timestep 41
timestep 41
timestep 42
timestep 43
timestep 44
timestep 44
timestep 45
timestep 46
timestep 47
timestep 48
timestep 49
timestep 49
timestep 49
timestep 49
timestep 49
timestep 49
timestep 49
timestep 50
timestep 50
timestep 50
timestep 50
timestep 50
timestep 50
timestep 50
timestep 50
timestep 50
timestep 51
timestep 51
timestep 51
timestep 51
timestep 52
timestep 53
timestep 53
timestep 53
timestep 53
timestep 53
timestep 54
timestep 55
timestep 55
instance result 55 0.6254527167696247 0.05148172378540039 8 2.78612877182688 2.8035546899229713
example  24
policy path /scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/traine

In [9]:
len(out_ml)

19

In [4]:
avg_ml_steps

41.578947368421055

In [6]:
avg_ml_ogap

4.6661177627139105

In [5]:
print(misc_time)
print(classification_time)
print(branching_time)
print(classified_nodes)
print(branched_nodes)

0.03909912862275776
0.1859548844789204
19.83186455776817
73.57894736842105
41.578947368421055


In [45]:
0.25/94.74

0.002638800928857927

In [34]:
18.27/0.238

76.76470588235294

In [33]:
1.50/6.3

0.2380952380952381

In [32]:
0.021/9.8

0.002142857142857143

In [41]:
ml = misc_time + classification_time + branching_time
ml

25.704898111025493

In [42]:
oracle_time/ml

29.35129180587768

In [43]:
oracle_time

754.4719652970632

In [8]:
6.92/17.83

0.3881099270891756

In [7]:
0.081/31

0.002612903225806452

In [12]:
ml_timesteps = np.mean(np.array([out_ml[i][3] for i in range(len(out_ml))]))
ml_timesteps

0.25096036659532556

In [9]:
Model ogap: 1.8025486716281995, time speedup: 128.31639072673866
BB time: 60.44540364742279, ML time: 7.381840499242147 
        
Model ogap: 0.95
ML time: 8.3
    
Omar = 3.8
Ogap = 2.70

142.85714285714286

In [13]:
60.44/7.38

8.18970189701897

In [14]:
8.18/6.99*7.38

8.636394849785408

In [15]:
60.44/3.8

15.905263157894737